In [1]:
%load_ext autoreload

In [2]:
autoreload 2

In [3]:
import sys
# !{sys.executable} -m pip install -r ../../requirements.txt

In [4]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

helper_path = os.path.abspath(os.path.join('../helper'))
if helper_path not in sys.path:
    sys.path.append(helper_path)

from helper.dataclass import HDDDataset
from helper.models.mymodel import MyModel
from helper.models.XCL import WeightedXCL
from helper.saver import *
from helper.preprocessing import *
from helper.metrics import *
from helper.eda import *
from sklearn.model_selection import train_test_split
from helper.metrics import *
from helper.saver import Saver

## Data preprocessing

In [5]:
hdd_dataset = HDDDataset.read_csv('ST14000NM001G.csv')

In [6]:
preprocessor = Preprocessing(hdd_dataset)
preprocessor.clear_unused_data()
preprocessor.add_target_column()
preprocessor.prepare_train_df()
#preprocessor.train_test_val_split(smote_param=0.1, smote_alg='enn')
preprocessor.train_test_val_split(oversampling='Borderline', sampling_strat=0.01)
preprocessor.add_time_features()
preprocessor.normalize_data(method='YJ', inplace=True)

Clearing unused columns...
Adding target column...
Prepairing train dataset...
Splitting train dataset...
Overampling with BorderlineSMOTE...


c:\Users\rocke\AppData\Local\Programs\Python\Python310\lib\site-packages\imblearn\over_sampling\_smote\filter.py:197: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


Adding time features to splitted dataset...
Normalizing splitted dataset with YJ...


## Training

In [9]:
%%time
model = MyModel('XGBoost')
model.fit(preprocessor.X_train, preprocessor.y_train)

=== Оптимизация гиперпараметров для XGBoost... ===


c:\Users\rocke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 12 is smaller than n_iter=50. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Лучшие параметры для XGBoost:
{'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1}
CPU times: total: 3h 46min 11s
Wall time: 23min 20s


## Testing

In [10]:
preds = model.predict(preprocessor.X_test)
metrics = Metrics.get_metrics(preds, preprocessor.y_test)
print(metrics)

{'accuracy': 0.9996038337959826, 'precision': 0.917910447761194, 'recall': 0.4785992217898833, 'f1': 0.6291560102301791, 'auc_roc': 0.739284573342071}


## Logging

In [11]:
Saver.save(model, preprocessor, metrics, save_csv=False)

c:\Users\rocke\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:50:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
